Apply P3INN on Core 2 data to obtain many quantiles for it. FINN is then trained on these quantiles (Data-SPAN) to obtain many retardations. Including to the retardations, FINN will compute the C field for each quantile. Then I also have a UQ estimate for C.

In [1]:
import shutil
import importlib
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import lib
import p3inn

importlib.reload(lib)
importlib.reload(p3inn)

<module 'p3inn' from '/Users/r/Documents/stud/ss24/finnpinn/src/p3inn/__init__.py'>

# Run

In [2]:
data_out_root = Path("../data_out").resolve()
assert data_out_root.exists()
pi3nn_root = data_out_root / "p3inn"
finn_root = data_out_root / "finn"
pi3nn_root.mkdir(exist_ok=True, parents=True)
finn_root.mkdir(exist_ok=True, parents=True)

In [3]:
data_core2_df = lib.load_exp_data(name="Core 2")
display(data_core2_df.head())

,time,c_diss
0,0.000000,0.000000
1,0.838329,0.000096
2,1.377912,0.000062
3,1.879996,0.000061
4,2.379996,0.000052


### PI3NN on experimental data to obtain PIs for different quantiles

In [4]:
p3inn_core2_dir = p3inn.P3innDir(pi3nn_root / "core2")

### FINN on all quantiles to obtain retardations

In [5]:
finn_fullspan_dir = finn_root / "core2_fullspan"
finn_fullspan_dir

PosixPath('/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/core2_fullspan')

In [6]:
conf_core2 = lib.load_exp_conf(name="Core 2")
display(conf_core2)

{'D': 2.0045664000000002e-05,
 'por': 0.288,
 'rho_s': 1957.0,
 'X': 0.026034999999999996,
 'T': 39.824440010000004,
 'Nx': 20.0,
 'Nt': 55.0,
 'sample_radius': 0.02375,
 'Q': 0.00010435199999999999,
 'solubility': 1.6,
 'Dirichlet': 0.0,
 'Cauchy': 1.0}

In [7]:
from pathlib import Path

# Prepare a list of commands to run in parallel
commands = []


for run in range(4):
    for quantile, bound_up, bound_down in p3inn_core2_dir.iter_pred_PIs():
        for bound_type, bound in [("up", bound_up), ("down", bound_down)]:
            bound_file = Path(
                f"/tmp/bound_{bound_type}_{quantile:g}.npy"
            )  # Save bound data to file
            np.save(bound_file, bound)

            finn_dir = lib.FinnDir(finn_fullspan_dir / f"{run}_{bound_type}_{quantile:g}")
            # print(finn_dir.path.resolve())

            command = f"python3 run_finn_core2_no_restarts.py {finn_dir.path.resolve()} {bound_file} > ../data_out/logs/{run}_{quantile:g}_{bound_type}.txt"
            commands.append(command)

In [8]:
# Write all commands to a temporary file (input for GNU Parallel)
commands_file = Path("/tmp/parallel_commands.txt")
with open(commands_file, "w") as f:
    f.write("\n".join(commands))
print(commands_file.resolve())

/private/tmp/parallel_commands.txt


In [9]:
command = f"cat {commands_file} | parallel -j 6 --bar"
# subprocess.run(command, check=True, shell=True)
print(command)

cat /tmp/parallel_commands.txt | parallel -j 6 --bar


In [10]:
!cat /tmp/parallel_commands.txt

python3 run_finn_core2_no_restarts.py /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/core2_fullspan/0_up_0.05 /tmp/bound_up_0.05.npy > ../data_out/logs/0_0.05_up.txt
python3 run_finn_core2_no_restarts.py /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/core2_fullspan/0_down_0.05 /tmp/bound_down_0.05.npy > ../data_out/logs/0_0.05_down.txt
python3 run_finn_core2_no_restarts.py /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/core2_fullspan/0_up_0.1 /tmp/bound_up_0.1.npy > ../data_out/logs/0_0.1_up.txt
python3 run_finn_core2_no_restarts.py /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/core2_fullspan/0_down_0.1 /tmp/bound_down_0.1.npy > ../data_out/logs/0_0.1_down.txt
python3 run_finn_core2_no_restarts.py /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/core2_fullspan/0_up_0.125 /tmp/bound_up_0.125.npy > ../data_out/logs/0_0.125_up.txt
python3 run_finn_core2_no_restarts.py /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/core2_fullspan/0_down_0.125 /tmp/bound_down_0.